# Logistic Regression

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# For reproducibility
torch.manual_seed(1) 

In [3]:
x_data = [[1,2], [2,3], [3,1], [4,3], [5,3], [6,2]]
y_data = [ [0],   [0],   [0],   [1],   [1],   [1] ]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
x_train.shape, y_train.shape

(torch.Size([6, 2]), torch.Size([6, 1]))

In [4]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b))) 
# hypothesis = torch.sigmoid(torch.matmul(x_train, W) + b)

In [5]:
hypothesis, hypothesis.shape

(tensor([[0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000],
         [0.5000]], grad_fn=<MulBackward0>),
 torch.Size([6, 1]))

In [6]:
losses = -(y_train * torch.log(hypothesis) +
           (1 - y_train) * torch.log(1 - hypothesis))
losses

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [7]:
cost = losses.mean()
cost

tensor(0.6931, grad_fn=<MeanBackward0>)

In [8]:
F.binary_cross_entropy(hypothesis, y_train) 

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [9]:
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    hypothesis = torch.sigmoid(torch.matmul(x_train, W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [11]:
W, b

(tensor([[3.2530],
         [1.5179]], requires_grad=True),
 tensor([-14.4819], requires_grad=True))

In [12]:
hypothesis = torch.sigmoid(torch.matmul(x_train, W) + b)
hypothesis

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward>)

In [13]:
prediction = hypothesis >= torch.FloatTensor([0.5])
prediction

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

In [14]:
correct_prediction = prediction.float() == y_train
correct_prediction

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])

## Higher Implementation with Class

In [15]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [19]:
model = BinaryClassifier()
optimizer = optim.SGD(model.parameters(), lr=1)

In [20]:
nb_epochs = 100
for epoch in range(nb_epochs+1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/100 Cost: 0.614994 Accuracy 66.67%
Epoch   10/100 Cost: 0.747549 Accuracy 83.33%
Epoch   20/100 Cost: 0.633216 Accuracy 83.33%
Epoch   30/100 Cost: 0.538123 Accuracy 83.33%
Epoch   40/100 Cost: 0.450406 Accuracy 83.33%
Epoch   50/100 Cost: 0.366382 Accuracy 83.33%
Epoch   60/100 Cost: 0.287368 Accuracy 83.33%
Epoch   70/100 Cost: 0.219289 Accuracy 83.33%
Epoch   80/100 Cost: 0.173225 Accuracy 100.00%
Epoch   90/100 Cost: 0.151674 Accuracy 100.00%
Epoch  100/100 Cost: 0.140280 Accuracy 100.00%
